In [1]:
import copy
import re

In [2]:
flask_prefixes = {
    "# to Maximum Charges": [
        ['Wide', '2', '+(16–19) to Maximum Charges'],
        ['Plentiful', '22', '+(20–23) to Maximum Charges'],
        ['Bountiful', '42', '+(24–27) to Maximum Charges'],
        ['Abundant', '62', '+(28–31) to Maximum Charges'],
        ['Ample', '82', '+(32–35) to Maximum Charges']],
    "#% increased Charge Recovery": [
        ['Constant', '3', '(16–20)% increased Charge Recovery'],
        ['Continuous', '23', '(21–25)% increased Charge Recovery'],
        ['Endless', '43', '(26–30)% increased Charge Recovery'],
        ['Bottomless', '63', '(31–45)% increased Charge Recovery'],
        ['Perpetual', '83', '(46–50)% increased Charge Recovery']],
    "#% increased Charge Recovery #% reduced effect": [
        ['Doled', '20', '(37–42)% increased Charge Recovery 25% reduced effect'],
        ['Provisioned', '36', '(43–48)% increased Charge Recovery 25% reduced effect'],
        ['Measured', '52', '(49–54)% increased Charge Recovery 25% reduced effect'],
        ['Allocated', '68', '(55–60)% increased Charge Recovery 25% reduced effect'],
        ['Rationed', '84', '(61–66)% increased Charge Recovery 25% reduced effect']],
    "#% chance to gain a Flask Charge when you deal a Critical Strike": [
        ["Medic's", '8', '(11–15)% chance to gain a Flask Charge when you deal a Critical Strike'],
        ["Physician's", '26', '(16–20)% chance to gain a Flask Charge when you deal a Critical Strike'],
        ["Doctor's", '44', '(21–25)% chance to gain a Flask Charge when you deal a Critical Strike'],
        ["Specialist's", '62', '(26–30)% chance to gain a Flask Charge when you deal a Critical Strike'],
        ["Surgeon's", '80', '(31–35)% chance to gain a Flask Charge when you deal a Critical Strike']],
    "Gain # Charge when you are Hit by an Enemy": [
        ["Transgressor's", '29', 'Gain 1 Charge when you are Hit by an Enemy'],
        ["Masochist's", '63', 'Gain 2 Charges when you are Hit by an Enemy'],
        ["Flagellant's", '80', 'Gain 3 Charges when you are Hit by an Enemy']],
    "#% reduced Duration #% increased effect": [
        ["Abecedarian's", '20', '(33–38)% reduced Duration 25% increased effect'],
        ["Dabbler's", '50', '(28–32)% reduced Duration 25% increased effect'],
        ["Alchemist's", '80', '(23–27)% reduced Duration 25% increased effect']],
    "#% increased Duration":[
        ["Investigator's", '20', '(16–20)% increased Duration'],
        ["Analyst's", '36', '(21–25)% increased Duration'],
        ["Examiner's", '52', '(26–30)% increased Duration'],
        ["Clinician's", '68', '(31–35)% increased Duration'],
        ["Experimenter's", '84', '(36–40)% increased Duration']],
    "#% reduced Charges per use":[
        ["Apprentice's", '14', '(14–16)% reduced Charges per use'],
        ["Scholar's", '31', '(17–19)% reduced Charges per use'],
        ["Practitioner's", '48', '(20–22)% reduced Charges per use'],
        ["Brewer's", '65', '(23–25)% reduced Charges per use'],
        ["Chemist's", '82', '(26–28)% reduced Charges per use']]
}

flask_suffixes = {
    "#% increased Armour during Flask effect": [
        ['of the Abalone', '6', '(41–45)% increased Armour during Flask effect'],
        ['of the Tortoise', '32', '(46–50)% increased Armour during Flask effect'],
        ['of the Pangolin', '58', '(51–55)% increased Armour during Flask effect'],
        ['of the Armadillo', '84', '(56–60)% increased Armour during Flask effect']],
    "#% increased Evasion Rating during Flask effect": [
        ['of the Gazelle', '6', '(41–45)% increased Evasion Rating during Flask effect'],
        ['of the Antelope', '32', '(46–50)% increased Evasion Rating during Flask effect'],
        ['of the Ibex', '58', '(51–55)% increased Evasion Rating during Flask effect'],
        ['of the Impala', '84', '(56–60)% increased Evasion Rating during Flask effect']],
    "#% increased Movement Speed during Flask effect": [
        ['of the Hare', '5', '(6–8)% increased Movement Speed during Flask effect'],
        ['of the Lynx', '65', '(9–11)% increased Movement Speed during Flask effect'],
        ['of the Cheetah', '85', '(12–14)% increased Movement Speed during Flask effect']],
    "#% increased Block and Stun Recovery during Flask effect": [
        ['of Stiffness', '1', '(51–56)% increased Block and Stun Recovery during Flask effect'],
        ['of Bracing', '19', '(57–62)% increased Block and Stun Recovery during Flask effect'],
        ['of Ballast', '37', '(63–68)% increased Block and Stun Recovery during Flask effect'],
        ['of Counterpoise', '55', '(69–74)% increased Block and Stun Recovery during Flask effect'],
        ['of Stabilisation', '73', '(75–80)% increased Block and Stun Recovery during Flask effect']],
    "#% additional Elemental Resistances during Flask effect": [
        ['of the Crystal', '1', '(21–24)% additional Elemental Resistances during Flask effect'],
        ['of the Prism', '21', '(25–28)% additional Elemental Resistances during Flask effect'],
        ['of the Kaleidoscope', '41', '(29–32)% additional Elemental Resistances during Flask effect'],
        ['of Variegation', '61', '(33–36)% additional Elemental Resistances during Flask effect'],
        ['of the Rainbow', '81', '(37–40)% additional Elemental Resistances during Flask effect']],
    "#% of Attack Damage Leeched as Life during Flask effect": [
        ['of Bloodshed', '10', '0.4% of Attack Damage Leeched as Life during Flask effect'],
        ['of Gore', '20', '0.5% of Attack Damage Leeched as Life during Flask effect'],
        ['of Carnage', '40', '0.6% of Attack Damage Leeched as Life during Flask effect'],
        ['of Butchery', '60', '0.7% of Attack Damage Leeched as Life during Flask effect'],
        ['of Bloodletting', '80', '0.8% of Attack Damage Leeched as Life during Flask effect']],
    "#% of Spell Damage Leeched as Energy Shield during Flask effect": [
        ['of Diverting', '10', '0.4% of Spell Damage Leeched as Energy Shield during Flask effect'],
        ['of Depletion', '20', '0.5% of Spell Damage Leeched as Energy Shield during Flask effect'],
        ['of Tapping', '40', '0.6% of Spell Damage Leeched as Energy Shield during Flask effect'],
        ['of Siphoning', '60', '0.7% of Spell Damage Leeched as Energy Shield during Flask effect'],
        ['of Draining', '80', '0.8% of Spell Damage Leeched as Energy Shield during Flask effect']],
    "#% increased Attack Speed during Flask effect": [
        ['of the Falcon', '12', '(9–11)% increased Attack Speed during Flask effect'],
        ['of the Eagle', '62', '(12–14)% increased Attack Speed during Flask effect'],
        ['of the Dove', '82', '(15–17)% increased Attack Speed during Flask effect']],
    "#% increased Cast Speed during Flask effect": [
        ['of the Albatross', '12', '(9–11)% increased Cast Speed during Flask effect'],
        ['of the Hummingbird', '62', '(12–14)% increased Cast Speed during Flask effect'],
        ['of the Horsefly', '82', '(15–17)% increased Cast Speed during Flask effect']],
    "#% increased Critical Strike Chance during Flask Effect": [
        ['of Stinging', '18', '(26–31)% increased Critical Strike Chance during Flask Effect'],
        ['of Piercing', '34', '(32–37)% increased Critical Strike Chance during Flask Effect'],
        ['of Rupturing', '50', '(38–43)% increased Critical Strike Chance during Flask Effect'],
        ['of Penetrating', '66', '(44–49)% increased Critical Strike Chance during Flask Effect'],
        ['of Incision', '82', '(50–55)% increased Critical Strike Chance during Flask Effect']],
    "#% chance to Freeze, Shock and Ignite during Flask effect": [
        ['of Foisting', '12', '(19–22)% chance to Freeze, Shock and Ignite during Flask effect'],
        ['of Imposing', '32', '(23–26)% chance to Freeze, Shock and Ignite during Flask effect'],
        ['of Wreaking', '52', '(27–30)% chance to Freeze, Shock and Ignite during Flask effect'],
        ['of Infliction', '72', '(31–34)% chance to Freeze, Shock and Ignite during Flask effect']],
    "#% Chance to Avoid being Stunned during Flask Effect": [
        ['of Composure', '12', '(31–35)% Chance to Avoid being Stunned during Flask Effect'],
        ['of Surefootedness', '29', '(36–40)% Chance to Avoid being Stunned during Flask Effect'],
        ['of Persistence', '46', '(41–45)% Chance to Avoid being Stunned during Flask Effect'],
        ['of Relentlessness', '63', '(46–50)% Chance to Avoid being Stunned during Flask Effect'],
        ['of Tenaciousness', '80', '(51–55)% Chance to Avoid being Stunned during Flask Effect']],
    "#% reduced Effect of Chill on you during Flask Effect #% reduced Freeze Duration on you during Flask Effect": [
        ['of the Rabbit', '4', '(36–41)% reduced Effect of Chill on you during Flask Effect (36–41)% reduced Freeze Duration on you during Flask Effect'],
        ['of the Cat', '23', '(42–47)% reduced Effect of Chill on you during Flask Effect (42–47)% reduced Freeze Duration on you during Flask Effect'],
        ['of the Fox', '42', '(48–52)% reduced Effect of Chill on you during Flask Effect (48–52)% reduced Freeze Duration on you during Flask Effect'],
        ['of the Sable', '61', '(52–59)% reduced Effect of Chill on you during Flask Effect (52–59)% reduced Freeze Duration on you during Flask Effect'],
        ['of the Bear', '80', '(60–65)% reduced Effect of Chill on you during Flask Effect (60–65)% reduced Freeze Duration on you during Flask Effect']],
    "#% reduced Effect of Shock on you during Flask Effect": [
        ['of the Plover', '6', '(36–41)% reduced Effect of Shock on you during Flask Effect'],
        ['of the Sandpiper', '25', '(42–47)% reduced Effect of Shock on you during Flask Effect'],
        ['of the Cormorant', '44', '(48–52)% reduced Effect of Shock on you during Flask Effect'],
        ['of the Sanderling', '63', '(52–59)% reduced Effect of Shock on you during Flask Effect'],
        ['of the Heron', '82', '(60–65)% reduced Effect of Shock on you during Flask Effect']],
    "#% reduced Effect of Curses on you during Flask Effect": [
        ['of the Petrel', '8', '(36–41)% reduced Effect of Curses on you during Flask Effect'],
        ['of the Mockingbird', '27', '(42–47)% reduced Effect of Curses on you during Flask Effect'],
        ['of the Curlew', '46', '(48–52)% reduced Effect of Curses on you during Flask Effect'],
        ['of the Kakapo', '65', '(52–59)% reduced Effect of Curses on you during Flask Effect'],
        ['of the Owl', '84', '(60–65)% reduced Effect of Curses on you during Flask Effect']],
    "#% chance to Avoid being Chilled during Flask Effect #% chance to Avoid being Frozen during Flask Effect": [
        ['of the Orca', '4', '(31–35)% chance to Avoid being Chilled during Flask Effect (31–35)% chance to Avoid being Frozen during Flask Effect'],
        ['of the Sea Lion', '23', '(36–40)% chance to Avoid being Chilled during Flask Effect (36–40)% chance to Avoid being Frozen during Flask Effect'],
        ['of the Narwhal', '42', '(41–45)% chance to Avoid being Chilled during Flask Effect (41–45)% chance to Avoid being Frozen during Flask Effect'],
        ['of the Beluga', '61', '(46–50)% chance to Avoid being Chilled during Flask Effect (46–50)% chance to Avoid being Frozen during Flask Effect'],
        ['of the Seal', '80', '(51–55)% chance to Avoid being Chilled during Flask Effect (51–55)% chance to Avoid being Frozen during Flask Effect']],
    "#% chance to Avoid being Ignited during Flask Effect": [
        ['of the Guppy', '6', '(31–35)% chance to Avoid being Ignited during Flask Effect'],
        ['of the Goldfish', '25', '(36–40)% chance to Avoid being Ignited during Flask Effect'],
        ['of the Carp', '44', '(41–45)% chance to Avoid being Ignited during Flask Effect'],
        ['of the Catfish', '63', '(46–50)% chance to Avoid being Ignited during Flask Effect'],
        ['of the Sunfish', '82', '(51–55)% chance to Avoid being Ignited during Flask Effect']],
    "#% chance to Avoid being Shocked during Flask Effect": [
        ['of Tree Moss', '6', '(31–35)% chance to Avoid being Shocked during Flask Effect'],
        ['of Turf Moss', '25', '(36–40)% chance to Avoid being Shocked during Flask Effect'],
        ['of Tooth Moss', '44', '(41–45)% chance to Avoid being Shocked during Flask Effect'],
        ['of Plume Moss', '63', '(46–50)% chance to Avoid being Shocked during Flask Effect'],
        ['of Bog Moss', '82', '(51–55)% chance to Avoid being Shocked during Flask Effect']],
    "#% less Duration Immunity to Poison during Flask Effect": [
        ['of the Skunk', '16', '(45–49)% less Duration Immunity to Poison during Flask Effect'],
        ['of the Hedgehog', '46', '(40–44)% less Duration Immunity to Poison during Flask Effect'],
        ['of the Opossum', '76', '(35–39)% less Duration Immunity to Poison during Flask Effect']],
    "#% less Duration Immunity to Shock during Flask Effect": [
        ['of the Conger', '6', '(45–49)% less Duration Immunity to Shock during Flask Effect'],
        ['of the Moray', '40', '(40–44)% less Duration Immunity to Shock during Flask Effect'],
        ['of the Eel', '74', '(35–39)% less Duration Immunity to Shock during Flask Effect']],
    "#% less Duration Immunity to Freeze and Chill during Flask Effect": [
        ['of the Deer', '4', '(45–49)% less Duration Immunity to Freeze and Chill during Flask Effect'],
        ['of the Walrus', '38', '(40–44)% less Duration Immunity to Freeze and Chill during Flask Effect'],
        ['of the Penguin', '72', '(35–39)% less Duration Immunity to Freeze and Chill during Flask Effect']],
    "#% less Duration Immunity to Ignite during Flask effect Removes Burning on use": [
        ['of the Urchin', '6', '(45–49)% less Duration Immunity to Ignite during Flask effect Removes Burning on use'],
        ['of the Mussel', '40', '(40–44)% less Duration Immunity to Ignite during Flask effect Removes Burning on use'],
        ['of the Starfish', '74', '(35–39)% less Duration Immunity to Ignite during Flask effect Removes Burning on use']],
    "#% less Duration Immunity to Bleeding and Corrupted Blood during Flask Effect": [
        ['of the Lizard', '8', '(45–49)% less Duration Immunity to Bleeding and Corrupted Blood during Flask Effect'],
        ['of the Skink', '42', '(40–44)% less Duration Immunity to Bleeding and Corrupted Blood during Flask Effect'],
        ['of the Iguana', '76', '(35–39)% less Duration Immunity to Bleeding and Corrupted Blood during Flask Effect']]
}

utility_flasks = [
    'Diamond',
    'Ruby',
    'Sapphire',
    'Topaz',
    'Granite',
    'Quicksilver',
    'Amethyst',
    'Quartz',
    'Jade',
    'Basalt',
    'Aquamarine',
    'Stibnite',
    'Sulphur',
    'Silver',
    'Bismuth',
    'Gold',
    'Corundum',
    'Iron'
]

flask_exceptions = {
    "of the Cat": "at$",
    "Gold": "^go",
    "#% increased Charge Recovery": "very$",
    "Silver": "^si",
    "Chemist's": "^ch"
}

In [3]:
def get_strings(flask_prefixes, flask_suffixes, utility_flasks, flask_exceptions, include_flask_name=True):
    all_strings = []
    utility_flasks = [flask for flask in utility_flasks if flask not in flask_exceptions]
    if include_flask_name:
        all_strings += utility_flasks
    def flask_affix_strings(affixes, all_strings):
        for elem in affixes: 
            n = len(affixes[elem])
            if elem not in flask_exceptions:
                all_strings.append(elem)
            for i in range(n):
                elem_name = affixes[elem][i][0]
                if elem_name not in flask_exceptions:
                    all_strings.append(elem_name)     
    flask_affix_strings(flask_prefixes, all_strings)
    flask_affix_strings(flask_suffixes, all_strings)
    return all_strings

In [4]:
flask_strings = get_strings(flask_prefixes, flask_suffixes, utility_flasks, flask_exceptions)

In [5]:
flask_strings

['Diamond',
 'Ruby',
 'Sapphire',
 'Topaz',
 'Granite',
 'Quicksilver',
 'Amethyst',
 'Quartz',
 'Jade',
 'Basalt',
 'Aquamarine',
 'Stibnite',
 'Sulphur',
 'Bismuth',
 'Corundum',
 'Iron',
 '# to Maximum Charges',
 'Wide',
 'Plentiful',
 'Bountiful',
 'Abundant',
 'Ample',
 'Constant',
 'Continuous',
 'Endless',
 'Bottomless',
 'Perpetual',
 '#% increased Charge Recovery #% reduced effect',
 'Doled',
 'Provisioned',
 'Measured',
 'Allocated',
 'Rationed',
 '#% chance to gain a Flask Charge when you deal a Critical Strike',
 "Medic's",
 "Physician's",
 "Doctor's",
 "Specialist's",
 "Surgeon's",
 'Gain # Charge when you are Hit by an Enemy',
 "Transgressor's",
 "Masochist's",
 "Flagellant's",
 '#% reduced Duration #% increased effect',
 "Abecedarian's",
 "Dabbler's",
 "Alchemist's",
 '#% increased Duration',
 "Investigator's",
 "Analyst's",
 "Examiner's",
 "Clinician's",
 "Experimenter's",
 '#% reduced Charges per use',
 "Apprentice's",
 "Scholar's",
 "Practitioner's",
 "Brewer's",
 '#%

In [6]:
def substring_lower(strings):
    return [mod.lower() for mod in strings]

def match_substring_to_fullstring(substring, string_list):
    for string in string_list:
        if substring in string:
            return string
    return "String not found."
        
def populate_substring_list(strings, string_len):
    strings = substring_lower(strings)
    curr_substrings = []
    for string in strings:
        curr_substring = ""
        lst = []
        i = 0
        while curr_substring != string[-string_len:]:
            curr_substring = string[i:i+string_len]
            lst.append(curr_substring)
            i += 1
        curr_substrings.append(lst)
    return curr_substrings

def generate_unique_substring(strings, flask_exceptions):
    mod_to_substring_dict = {}
    string_len = 3
    while len(mod_to_substring_dict) < len(strings):
        substring_lists = populate_substring_list(strings, string_len)

        num_substring_lists = len(substring_lists)
        for x in range(num_substring_lists):
            curr_substring_list = substring_lists[x]
            for substring in curr_substring_list:
                fullstring = match_substring_to_fullstring(substring, substring_lower(strings))
                if fullstring in mod_to_substring_dict:
                    continue
                substring_has_dupe = False
                for y in range(num_substring_lists): 
                    compare_substring_list = substring_lists[y]
                    if compare_substring_list == curr_substring_list:
                        continue
                    if substring in compare_substring_list:
                        substring_has_dupe = True
                        continue
                if substring_has_dupe == False:
                    # skip over 3 char substrings with space
                    if len(substring) == 3 and " " in substring:
                        continue
                    if "x%" in substring:
                        substring = substring.replace("x%", "\d+%")
                    mod_to_substring_dict[fullstring] = substring
                    break
        string_len += 1
    flask_exceptions_lower = dict((k.lower(), v) for k, v in flask_exceptions.items())
    mod_to_substring_dict.update(flask_exceptions_lower)
    return mod_to_substring_dict

In [26]:
substring_dict = generate_unique_substring(flask_strings, flask_exceptions)

In [8]:
def is_substring(strings):
    strings = [string.lower() for string in strings]
    for string in strings: 
        if sum(string in s for s in strings) >=2:
            print(string)

is_substring(flask_strings)

In [9]:
#TODO: match affix list with substring
def flask_affix_with_substring(affixes, substring_dict):
    affixes_with_substring = copy.deepcopy(affixes)
    for elem in affixes_with_substring:
        n = len(affixes_with_substring[elem])
        for i in range(n):
            elem_name = affixes_with_substring[elem][i][0].lower()
            substring = substring_dict[elem_name]
            affixes_with_substring[elem][i].append(substring)
    return affixes_with_substring

flask_affix_with_substring_dict = flask_affix_with_substring(flask_prefixes, substring_dict)

In [16]:
#TODO: numeric tier match

def flask_affix_with_numeric_substring(flask_affix_with_substring_dict, substring_dict):
    flask_affix_names = list(flask_affix_with_substring_dict.keys())
    numeric_substring_dict = copy.deepcopy(flask_affix_with_substring_dict)
    for affix_name in flask_affix_names:
        substring = substring_dict[affix_name.lower()]

        # check if ascending or descending
        first_elem = numeric_substring_dict[affix_name][0][2]
        second_elem = numeric_substring_dict[affix_name][1][2]

        first_elem_num = re.findall("\d\.?\d?", first_elem)
        second_elem_num = re.findall("\d\.?\d?", second_elem)

        ascending = True if (first_elem_num < second_elem_num) else False

        decimal = True if ("." in first_elem_num) else False

        # loop for each tier
        for tier in numeric_substring_dict[affix_name]:
            num_val = float(re.findall("\d\.?\d?", tier[2])[0])
            # decimal tiers are always ascending
            if decimal: 
                x, y = 0, int(num_val*10)
                numeric_substring = f"(0\.{y}|0\.{y+1}).*"
            elif ascending: 
                # x, y are digits of min value of specific tier with ascending values; does need to include single digits
                x, y = int(num_val//10), int(num_val%10)
                # double digit ascending
                if x > 0:
                    numeric_substring = f"({x}[{y}-9]|[{x+1}-9]\d).*"
                #single digit ascending
                else:
                    numeric_substring = f"([{y}-9]|\d\d).*"
            else:
                # a, b are digts of max value of tier with descending values; doesn't need to cover single digit numbers
                a, b = int(num_val//10), int(num_val%10)
                numeric_substring = f"({a}[0-{b}]|[1-{x-1}]\d).*"
            full_numeric_substring = numeric_substring+substring
            tier.append(full_numeric_substring)
    return numeric_substring_dict

In [27]:
# NOTE: returning will show repr \\, printing will show real single \
flask_affix_with_numeric_substring_dict = (
    flask_affix_with_numeric_substring(flask_affix_with_substring_dict, substring_dict))

print(flask_affix_with_numeric_substring_dict['# to Maximum Charges'][0])

['Wide', '2', '+(16–19) to Maximum Charges', 'wid', '(1[6-9]|[2-9]\\d).*max']


In [30]:
# TODO: substring letter tier match
def flask_affix_with_numeric_and_letter_substring(flask_affix_with_numeric_substring_dict):
    flask_affix_names = list(flask_affix_with_numeric_substring_dict.keys())
    letter_substring_dict = copy.deepcopy(flask_affix_with_numeric_substring_dict)
    for affix_name in flask_affix_names:
        # loop for each tier
        num_tiers = len(letter_substring_dict[affix_name])
        for i in range(num_tiers):
            curr_tier = letter_substring_dict[affix_name][i]
            tier_substring = curr_tier[3] 
            for j in range(i+1, num_tiers):
                j_tier = letter_substring_dict[affix_name][j]
                tier_substring += f"|{j_tier[3]}"
            curr_tier.append(tier_substring)
    return letter_substring_dict

flask_affix_with_numeric_and_letter_substring(flask_affix_with_numeric_substring_dict)


{'# to Maximum Charges': [['Wide',
   '2',
   '+(16–19) to Maximum Charges',
   'wid',
   '(1[6-9]|[2-9]\\d).*max',
   'wid|plen|bou|abu|amp'],
  ['Plentiful',
   '22',
   '+(20–23) to Maximum Charges',
   'plen',
   '(2[0-9]|[3-9]\\d).*max',
   'plen|bou|abu|amp'],
  ['Bountiful',
   '42',
   '+(24–27) to Maximum Charges',
   'bou',
   '(2[4-9]|[3-9]\\d).*max',
   'bou|abu|amp'],
  ['Abundant',
   '62',
   '+(28–31) to Maximum Charges',
   'abu',
   '(2[8-9]|[3-9]\\d).*max',
   'abu|amp'],
  ['Ample',
   '82',
   '+(32–35) to Maximum Charges',
   'amp',
   '(3[2-9]|[4-9]\\d).*max',
   'amp']],
 '#% increased Charge Recovery': [['Constant',
   '3',
   '(16–20)% increased Charge Recovery',
   'ons',
   '(1[6-9]|[2-9]\\d).*very$',
   'ons|ont|end|bot|rpe'],
  ['Continuous',
   '23',
   '(21–25)% increased Charge Recovery',
   'ont',
   '(2[1-9]|[3-9]\\d).*very$',
   'ont|end|bot|rpe'],
  ['Endless',
   '43',
   '(26–30)% increased Charge Recovery',
   'end',
   '(2[6-9]|[3-9]\\d).*very$'

In [15]:
#TODO: match open prefix/suffix